# INDICNER

In [1]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=e0a05e1c138878e6da1d43e1095d431e568844c33643a3134ca6902659ae00ec
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
!pip install accelerate -U
!pip install transformers[torch]

### LOADING THE DATASET

In [3]:
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

### EXPLORING THE DATASET

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 985787
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 867
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13460
    })
})

In [5]:
raw_datasets.column_names

{'train': ['tokens', 'ner_tags'],
 'test': ['tokens', 'ner_tags'],
 'validation': ['tokens', 'ner_tags']}

In [6]:
idx=0
rec=raw_datasets['train'][idx]
for w, t in zip(rec['tokens'],rec['ner_tags']):
  print('{}\t{}'.format(w,t))

सेक्टर	0
55/56	0
के	0
एसएचओ	0
अरविंद	1
कुमार	2
ने	0
बताया	0
कि	0
इस	0
मामले	0
में	0
आईपीसी	0
की	0
धारा	0
376	0
-	0
डी	0
(	0
गैंगरेप	0
)	0
के	0
तहत	0
मामला	0
दर्ज	0
कर	0
लिया	0
गया	0
है	0
।	0


In [7]:
column_names = raw_datasets["train"].column_names
print(column_names)

features = raw_datasets["train"].features
print(features)

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [8]:
text_column_name = "tokens"
label_column_name = "ner_tags"
features = raw_datasets["train"].features
label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)

{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [9]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/IndicNER', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/IndicNER', num_labels=num_labels )

2024-03-12 14:21:13.318224: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 14:21:13.318321: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 14:21:13.483551: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

### LOADING TRAIN DATASET (20000 LINES) AND VALIDATION DATASET

In [11]:
train_dataset = raw_datasets["train"]
train_dataset=train_dataset.select(range(20000))
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/5 [00:00<?, ?ba/s]

In [12]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #0:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #1:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/4 [00:00<?, ?ba/s]

In [14]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [15]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

### TRAINING THE MODEL

In [16]:
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy = "epoch",
    learning_rate=5e-5)

In [17]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

In [18]:
trainer.args

TrainingArguments(
_n_gpu=2,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_la

In [19]:
train_result = trainer.train()
metrics = train_result.metrics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,0.326300
1000,0.153100
1500,0.132800
2000,0.106200
2500,0.103200
3000,0.073900
3500,0.074200


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

In [21]:
# model.config

### EVALUATION METRICS

In [20]:
metrics = trainer.evaluate()

trainer.log_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        3.0
  eval_LOC_f1             =     0.8315
  eval_LOC_number         =      10213
  eval_LOC_precision      =     0.8116
  eval_LOC_recall         =     0.8523
  eval_ORG_f1             =     0.6783
  eval_ORG_number         =       9786
  eval_ORG_precision      =     0.6715
  eval_ORG_recall         =     0.6853
  eval_PER_f1             =     0.8122
  eval_PER_number         =      10568
  eval_PER_precision      =     0.8006
  eval_PER_recall         =     0.8242
  eval_loss               =     0.2078
  eval_overall_accuracy   =     0.9459
  eval_overall_f1         =     0.7761
  eval_overall_precision  =     0.7635
  eval_overall_recall     =     0.7891
  eval_runtime            = 0:05:11.69
  eval_samples_per_second =     43.183
  eval_steps_per_second   =      2.701


### SAVE THE MODEL

In [21]:
model.save_pretrained("NER_model")

In [22]:
# !zip -r file.zip /kaggle/working/NER_model

In [23]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

### CONFIGURING THE MODEL FOR PREDICITON OF TOKENS ON 25 SENTENCES FROM QUESTION 1

In [24]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}
print(id2label)
print(label2id)

{'0': 'O', '1': 'B-PER', '2': 'I-PER', '3': 'B-ORG', '4': 'I-ORG', '5': 'B-LOC', '6': 'I-LOC'}
{'O': '0', 'B-PER': '1', 'I-PER': '2', 'B-ORG': '3', 'I-ORG': '4', 'B-LOC': '5', 'I-LOC': '6'}


In [25]:
import json
config=json.load(open("/kaggle/working/NER_model/config.json"))

In [26]:
config["id2label"]=id2label
config["label2id"]=label2id

In [27]:
json.dump(config, open("/kaggle/working/NER_model/config.json","w"))

In [39]:
model = AutoModelForTokenClassification.from_pretrained("NER_model")

### FINDING PRECISION, RECALL AN F1-SCORE ON TEST SET

In [29]:
tokenized_test_set = {}

tokenized_test_set = raw_datasets['test'].map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=32,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset of language {0}".format(lang),
)

Running tokenizer on test dataset of language hi #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #4:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #3:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Running tokenizer on test dataset of language hi #27:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #28:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #29:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #30:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #31:   0%|          | 0/1 [00:00<?, ?ba/s]

In [30]:
predictions, labels, metrics = trainer.predict(tokenized_test_set, metric_key_prefix="hi")

lang_specific_results = {}
for key in metrics:
    if 'overall_precision' in key:
        lang_specific_results['Precision'] = metrics[key]
    elif 'overall_recall' in key:
        lang_specific_results['Recall'] = metrics[key]
    elif 'overall_f1' in key:
        lang_specific_results['F1'] = metrics[key]
final_metrics = lang_specific_results

In [31]:
print("METRICS FOR TEST SET PREDICTION BY MODEL: \n",final_metrics)

METRICS FOR TEST SET PREDICTION BY MODEL: 
 {'Precision': 0.7680190930787589, 'Recall': 0.8341109383100052, 'F1': 0.7997017892644135}


In [32]:
# print(raw_datasets["test"][1])

### CREATING DATASET OF 25 SENTENCES FROM QUESTION 1

In [33]:
from datasets import Dataset

data1 = {
    "tokens": [["इस", "बार", "गुरुवार", "को", "ईसाई", "समुदाय", "के", "लोगों", "ने", "यहां", "‘मुर्दों", "की", "ईद", "मनाई", "।"],
              ["उन्होंने", "कहा", ",", "डॉ", "राजेन्द्र", "प्रसाद", ",", "बाबा", "साहेब", "भीमराव", "अंबेडकर", ",", "सरदार", "पटेल", ",", "मौलाना", "आजाद", ",", "सुचेता", "कृपलानी", "और", "अनेक", "अनगिनत", "महापुरुषों", "ने", "प्रत्यक्ष", "और", "अप्रत्यक्ष", "योगदान", "देकर", "ये", "महान", "विरासत", "हमें", "सौंपी", "हैं", "।"],
              ["मुझे", "अपने", "विपक्षी", "मित्रों", "के", "तर्क", "इस", "वैज्ञानिक", "के", "निष्कर्ष", "के", "समान", "ही", "प्रतीत", "हो", "रहे", "हैं", "...."],
              ["पार्टी", "अगर", "कोई", "हुई", "तो", "कितनी", "फीकी", "लगेगी", "बिना", "गाने", "के", "!"],
              ["उनके", "अनुसार", "सरकारी", "अस्पतालों", "की", "तरह", "सरकारी", "स्कूलों", "की", "हालत", "बदतर", "हो", "चुकी", "है", "।"],
              ["देश", "में", "सभी", "प्रकार", "के", "छोटे-बड़े", "कृषि", ",", "सेवा", ",", "व्यपार", ",", "उद्योग", "का", "पंजीकरण", "तुरंत", "अनिवार्य", "होना", "चाहिए", "साथ", "उनको", "रु", "1000", "से", "ऊपर", "के", "लेन", "देन", "को", "ऑन लाइन", "/", "इलेक्ट्रॉनिक", "माध्यम", "से", "करना", "अनिवार्य", "होना", "चाहिये", "।"],
              ["इस", "सम्मेलन", "का", "उद्घाटन", "करने", "पहुंचे", "पीएम", "मोदी", "ने", "कहा", "कि", "टीबी", "को", "25", "साल", "पहले", "डब्ल्यूएचओं", "ने", "एमरजंसी", "घोषित", "किया", "था", "और", "तभी", "से", "इसके", "खिलाफ", "अभियान", "चल", "रहा", "है", "।"],
              ["इस", "वक्त", "खुदरा", "महंगाई", "दर", "भी", "पांच", "महीने", "के", "उच्चतम", "स्तर", "पर", "है", "।"],
              ["साल", "2007", "के", "क्रिकेट", "विश्व", "कप", "में", "भारत", "के", "खराब", "प्रदर्शन", "के", "बाद", "भी", "विज्ञापनों", "पर", "खर्च", "हुए", "6,500", "करोड़", "रुपये", "में", "से", "भी", "7-10", "फीसदी", "की", "हिस्सेदारी", "थी", "।"],
              ["हिन्दी", "के", "शिक्षण-प्रशिक्षण", "के", "लिए", "देश", "के", "कई", "हिस्सों", "के", "साथ-साथ", "विदेशों", "में", "सेंटर", "खोलने", "की", "भी", "योजना", "है", "।"],
              ["क्या", "गांधी", "के", "इस", "सुझाव", "पर", "विमर्श", "का", "समय", "आ", "गया", "है", "?", "राजनीतिक", "पार्टियां", "हमारी", "संवैधानिक", "आवश्यकता", "नहीं", "हैं", ".", "इसलिए", "उम्मीदवारों", "को", "निर्दलीय", "होने", "की", "छूट", "भी", "है", "."],
              ["अनायत", "इसके", "लिए", "विभागाध्यक्ष", "व", "छात्रों", "को", "बधाई", "दी", "है", "।"],
              ["इसके", "बाद", "इंटरनेशनल", "अलर्ट", "भी", "जारी", "कर", "दिया", "गया", "था", "।"],
              ["राज्य", "समन्वयक", "ने", "सीमा", "पुलिस", "के", "अधिकारियों", "को", "भेजे", "एक", "आदेश", "में", "“घोषित", "विदेशियों”", "के", "परिवार", "के", "सदस्यों", "को", "ट्रिब्यूनल", "के", "समक्ष", "प्रस्तुत", "करने", "को", "कहा", "."],
              ["जब", "बड़े-बड़े", "पापी-कपटी", "कामी-क्रोधी", "लोभी", "बदल", "सकते", "हैं", "तो", "हम", "क्यों", "नहीं", "बदल", "सकते", "।"],
              ["फिल्म", "में", "नायक", "हैं", "संजय", "खान", "और", "नायिका", "मुमताज", "और", "सहनायक", "फिरोज", "खान", "।", "रफी", "साहब", "आशा", "भोंसले", "और", "साथियो", "का", "गाया", "यह", "गीत", "पहले", "रेडियो", "के", "सभी", "केन्द्रों", "से", "बहुत", "सुनवाया", "जाता", "था", "।", "अब", "बहुत", "समय", "से", "नहीं", "सुना", "।", "इसके", "कुछ", "बोल", "याद", "आ", "रहे", "हैं", "-"],
              ["इसमें", "कहा", "गया", "कि", "'अकेले", "याचिकाकर्ता", "को", "दंडित", "नहीं", "किया", "जा", "रहा", "है", ",", "बल्कि", "आपराधिक", "कार्यवाही", "के", "कारण", "उसका", "पूरा", "परिवार", "अत्यंत", "पीड़ित", "हुआ", "है", "।"],
              ["हम", "आपकी", "सेवा", "में", "अपने", "शरीर", "के", "बारे", "में", "बहुत", "ही", "साधारण", "सी", "बात", "पेश", "करते", "हैं", "।"],
              ["बायर्न", "म्यूनिख", "में", "इस", "सीजन", "लोन", "पर", "शामिल", "हुए", "ब्राजील", "के", "खिलाड़ी", "फिलिप", "कोटिन्हो", "को", "कहना", "है", "कि", "वह", "जर्मन", "क्लब", "में", "लंबे", "समय", "तक", "रहकर", "कई", "ट्रॉफी", "जीतना", "चाहते", "हैं", "।"],
              ["गोवर्धन", "पूजा", "में", "भगवान", "को", "छप्पन", "भोग", "का", "प्रसाद", "चढ़ाया", "जाता", "हैं", "।"],
              ["कार्यवाहक", "कलक्टर", "ने", "आदेश", "जारी", "करते", "हुए", "विभागीय", "निर्देशानुसार", "निरीक्षण", "एवं", "व्यवस्था", "का", "मूल्यांकन", "नहीं", "करने", "पर", "छात्रावास", "में", "अनियमितताएं", "व्याप्त", "होने", "के", "कारण", "सहायक", "निदेशक", "दिलीप", "रोकडिय़ा", "को", "नोटिस", "जारी", "कर", "तीन", "दिन", "में", "जवाब", "देने", "को", "कहा", "है", "।"],
              ["हनीसकल", "की", "खेती", "क्षेत्र", "मुख्य", "रूप", "से", "शेडोंग", "शानक्सी", "हेनान", "हेबै", "हुबेई", "जियांग्सी", "गुआंग्डोंग", "और", "अन्य", "स्थानों", "पर", "केंद्रित", "हैं", "।"],
              ["इसके", "साथ", "ही", "चीन", "सक्रिय", "सेवा", "में", "विमानवाही", "पोत", "रखने", "वाला", "दुनिया", "का", "10वां", "और", "संयुक्त", "राष्ट्र", "सुरक्षा", "परिषद", "के-O", "स्थायी", "पांच", "सदस्यों", "में", "अंतिम", "देश", "बन", "गया", "है", "।"],
              ["संसारके", "दुःख", "मुझे", "प्राणप्रिय", "लगते", "हैं", ",", "उन्हें", "मैं", "कैसे", "छोड", ",", "सकता", "हूँ", "?", "उन", "दुःखोंको", "तो", "मैंने", "ही", "बुलाया", "है", "."],
              ["उन्होंने", "इस", "बाबत", "पिछले", "दिनों", "फेसबुक", "पर", "जो", "कुछ", "लाइनें", "लिखकर", "अपने", "जानने", "वालों", "को", "सूचित", "किया", "वह", "इस", "प्रकार", "है"]],
    "ner_tags": [[0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 1, 2, 0, 1, 2, 2, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 1, 2, 0, 0, 1, 0, 0, 1, 2, 0, 1, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [5, 6, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 1, 2, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
}

dataset = Dataset.from_dict(data1)

print(dataset)

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 25
})


### FINDING PRECISION, RECALL AND F1-SCORE FOR ABOVE SENTENCES

In [40]:
tokenized_test_set = {}

tokenized_test_set = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=32,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset of language {0}".format("hi"),
)

Running tokenizer on test dataset of language hi #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #3:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #4:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Running tokenizer on test dataset of language hi #20:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #21:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #23:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #24:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #22:   0%|          | 0/1 [00:00<?, ?ba/s]

In [41]:
predictions, labels, metrics = trainer.predict(tokenized_test_set, metric_key_prefix="hi")

lang_specific_results = {}
for key in metrics:
    if 'overall_precision' in key:
        lang_specific_results['Precision'] = metrics[key]
    elif 'overall_recall' in key:
        lang_specific_results['Recall'] = metrics[key]
    elif 'overall_f1' in key:
        lang_specific_results['F1'] = metrics[key]
final_metrics = lang_specific_results

In [42]:
print("METRICS FOR PREDICITON ON 25 SENTENCES FROM QUES1: \n",final_metrics)

METRICS FOR PREDICITON ON 25 SENTENCES FROM QUES1: 
 {'Precision': 0.6153846153846154, 'Recall': 0.5161290322580645, 'F1': 0.5614035087719298}


### PREDICTING NER-TAGS FOR 25 SENTENCES USING THE MODEL

In [43]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

In [44]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)
    
    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]
    
    predicted_labels = []
    
    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]
    
    return predicted_labels

In [45]:
sentences = ['इस बार गुरुवार को ईसाई समुदाय के लोगों ने यहां ‘मुर्दों की ईद’ मनाई।',
'उन्होंने कहा, डॉ राजेन्द्र प्रसाद, बाबा साहेब भीमराव अंबेडकर, सरदार पटेल, मौलाना आजाद, सुचेता कृपलानी और अनेक अनगिनत महापुरुषों ने प्रत्यक्ष और अप्रत्यक्ष योगदान देकर ये महान विरासत हमें सौंपी हैं।'
'मुझे अपने विपक्षी मित्रों के तर्क इस वैज्ञानिक के निष्‍कर्ष के समान ही प्रतीत हो रहे हैं....',
'पार्टी अगर कोई हुई तो कितनी फीकी लगेगी बिना गाने के!',
'उनके अनुसार सरकारी अस्पतालों की तरह सरकारी स्कूलों की हालत बदतर हो चुकी है।',
'देश में सभी प्रकार के छोटे-बड़े कृषि, सेवा, व्यपार,उद्योग का पंजीकरण तुरंत अनिवार्य होना चाहिए, साथ उनको रु 1000 से ऊपर के लेन देन को ऑन लाइन/ इलेक्ट्रॉनिक माध्यम से करना अनिवार्य होना चाहिये।',
'इस सम्मेलन का उद्घाटन करने पहुंचे पीएम मोदी ने कहा कि टीबी को 25 साल पहले डब्ल्यूएचओं ने एमरजंसी घोषित किया था और तभी से इसके खिलाफ अभियान चल रहा है।'
'इस वक्त खुदरा महंगाई दर भी पांच महीने के उच्चतम स्तर पर है।',
'साल 2007 के क्रिकेट विश्व कप में भारत के खराब प्रदर्शन के बाद भी विज्ञापनों पर खर्च हुए 6,500 करोड़ रुपये में से भी 7-10 फीसदी की हिस्सेदारी थी।',
'हिन्दी के शिक्षण-प्रशिक्षण के लिए देश के कई हिस्सों के साथ-साथ विदेशों में सेंटर खोलने की भी योजना है।',
'क्या गांधी के इस सुझाव पर विमर्श का समय आ गया है? राजनीतिक पार्टियां हमारी संवैधानिक आवश्यकता नहीं हैं. इसलिए उम्मीदवारों को निर्दलीय होने की छूट भी है.',
'अनायत इसके लिए विभागाध्यक्ष व छात्रों को बधाई दी है।',
'इसके बाद इंटरनेशनल अलर्ट भी जारी कर दिया गया था।',
'राज्य समन्वयक ने सीमा पुलिस के अधिकारियों को भेजे एक आदेश में “घोषित विदेशियों” के परिवार के सदस्यों को ट्रिब्यूनल के समक्ष प्रस्तुत करने को कहा.',
'जब बड़े-बड़े पापी-कपटी, कामी-क्रोधी, लोभी बदल सकते हैं, तो हम क्यों नहीं बदल सकते।',
'फिल्म में नायक हैं संजय खान और नायिका मुमताज और सहनायक फिरोज खान। रफी साहब, आशा भोंसले और साथियो का गाया यह गीत पहले रेडियो के सभी केन्द्रों से बहुत सुनवाया जाता था। अब बहुत समय से नहीं सुना। इसके कुछ बोल याद आ रहे हैं-',
'इसमें कहा गया कि अकेले याचिकाकर्ता को दंडित नहीं किया जा रहा है, बल्कि आपराधिक कार्यवाही के कारण उसका पूरा परिवार अत्यंत पीड़ित हुआ है।',
'हम आपकी सेवा में अपने शरीर के बारे में बहुत ही साधारण सी बात पेश करते हैं।',
'बायर्न म्यूनिख में इस सीजन लोन पर शामिल हुए ब्राजील के खिलाड़ी फिलिप कोटिन्हो को कहना है कि वह जर्मन क्लब में लंबे समय तक रहकर कई ट्रॉफी जीतना चाहते हैं।',
'गोवर्धन पूजा में भगवान को छप्पन भोग का प्रसाद चढ़ाया जाता हैं।',
'कार्यवाहक कलक्टर ने आदेश जारी करते हुए विभागीय निर्देशानुसार निरीक्षण एवं व्यवस्था का मूल्यांकन नहीं करने पर छात्रावास में अनियमितताएं व्याप्त होने के कारण सहायक निदेशक दिलीप रोकडिय़ा को नोटिस जारी कर तीन दिन में जवाब देने को कहा है।',
'हनीसकल की खेती क्षेत्र मुख्य रूप से शेडोंग, शानक्सी, हेनान, हेबै, हुबेई, जियांग्सी, गुआंग्डोंग और अन्य स्थानों पर केंद्रित हैं।',
'इसके साथ ही चीन सक्रिय सेवा में विमानवाही पोत रखने वाला दुनिया का 10वां और संयुक्त राष्ट्र सुरक्षा परिषद के स्थायी पांच सदस्यों में अंतिम देश बन गया है।',
'संसारके दुःख मुझे प्राणप्रिय लगते हैं, उन्हें मैं कैसे छोड. सकता हूँ ? उन दुःखोंको तो मैंने ही बुलाया है.',
'उन्होंने इस बाबत पिछले दिनों फेसबुक पर जो कुछ लाइनें लिखकर अपने जानने वालों को सूचित किया, वह इस प्रकार है']

In [46]:
predictions = []

for sentence in sentences:
    predicted_labels = get_predictions(sentence=sentence, 
                                       tokenizer=tokenizer,
                                       model=model
                                       )
    predict = []
    for index in range(len(sentence.split(' '))):
        predict.append(sentence.split(' ')[index] + ":" + predicted_labels[index])
    predictions.append(predict)
print(predictions)

[['इस:O', 'बार:O', 'गुरुवार:O', 'को:O', 'ईसाई:O', 'समुदाय:O', 'के:O', 'लोगों:O', 'ने:O', 'यहां:O', '‘मुर्दों:O', 'की:O', 'ईद’:O', 'मनाई।:O'], ['उन्होंने:O', 'कहा,:O', 'डॉ:O', 'राजेन्द्र:O', 'प्रसाद,:B-PER', 'बाबा:I-PER', 'साहेब:O', 'भीमराव:B-PER', 'अंबेडकर,:B-PER', 'सरदार:I-PER', 'पटेल,:I-PER', 'मौलाना:O', 'आजाद,:B-PER', 'सुचेता:I-PER', 'कृपलानी:O', 'और:B-PER', 'अनेक:I-PER', 'अनगिनत:O', 'महापुरुषों:B-PER', 'ने:I-PER', 'प्रत्यक्ष:O', 'और:O', 'अप्रत्यक्ष:O', 'योगदान:O', 'देकर:O', 'ये:O', 'महान:O', 'विरासत:O', 'हमें:O', 'सौंपी:O', 'हैं।मुझे:O', 'अपने:O', 'विपक्षी:O', 'मित्रों:O', 'के:O', 'तर्क:O', 'इस:O', 'वैज्ञानिक:O', 'के:O', 'निष्\u200dकर्ष:O', 'के:O', 'समान:O', 'ही:O', 'प्रतीत:O', 'हो:O', 'रहे:O', 'हैं....:O'], ['पार्टी:O', 'अगर:O', 'कोई:O', 'हुई:O', 'तो:O', 'कितनी:O', 'फीकी:O', 'लगेगी:O', 'बिना:O', 'गाने:O', 'के!:O'], ['उनके:O', 'अनुसार:O', 'सरकारी:O', 'अस्पतालों:O', 'की:O', 'तरह:O', 'सरकारी:O', 'स्कूलों:O', 'की:O', 'हालत:O', 'बदतर:O', 'हो:O', 'चुकी:O', 'है।:O'], ['देश:O', 'में:O', '

### CREATING LIST OF ABOVE TOKENS FOR EACH CLASS

In [48]:
pd = {'O' : [],'B_PER' : [],'I_PER' : [],'B_LOC' : [],'I_LOC' : [],'B_ORG' : [],'I_ORG' : [],'B_MISC' : [],'I_MISC' : []}

In [49]:
for sublist in predictions:
    for item in sublist:
        temp = item.split(':')
        token = temp[0]
        if(len(temp)>1):
            tag = temp[1]
            if(tag=='O'): pd['O'].append(token)
            elif(tag=='B-PER'): pd['B_PER'].append(token)
            elif(tag=='I-PER'): pd['I_PER'].append(token)
            elif(tag=='B-LOC'): pd['B_LOC'].append(token)
            elif(tag=='I-LOC'): pd['I_LOC'].append(token)
            elif(tag=='B-ORG'): pd['B_ORG'].append(token)
            elif(tag=='I-ORG'): pd['I_ORG'].append(token)
#             elif(tag=='B-MISC'): pd['B_MISC'].append(token)
#             elif(tag=='I-MISC'): pd['I_MISC'].append(token)
for key in pd.keys():
    print(key, ":", pd[key])

O : ['इस', 'बार', 'गुरुवार', 'को', 'ईसाई', 'समुदाय', 'के', 'लोगों', 'ने', 'यहां', '‘मुर्दों', 'की', 'ईद’', 'मनाई।', 'उन्होंने', 'कहा,', 'डॉ', 'राजेन्द्र', 'साहेब', 'मौलाना', 'कृपलानी', 'अनगिनत', 'प्रत्यक्ष', 'और', 'अप्रत्यक्ष', 'योगदान', 'देकर', 'ये', 'महान', 'विरासत', 'हमें', 'सौंपी', 'हैं।मुझे', 'अपने', 'विपक्षी', 'मित्रों', 'के', 'तर्क', 'इस', 'वैज्ञानिक', 'के', 'निष्\u200dकर्ष', 'के', 'समान', 'ही', 'प्रतीत', 'हो', 'रहे', 'हैं....', 'पार्टी', 'अगर', 'कोई', 'हुई', 'तो', 'कितनी', 'फीकी', 'लगेगी', 'बिना', 'गाने', 'के!', 'उनके', 'अनुसार', 'सरकारी', 'अस्पतालों', 'की', 'तरह', 'सरकारी', 'स्कूलों', 'की', 'हालत', 'बदतर', 'हो', 'चुकी', 'है।', 'देश', 'में', 'सभी', 'प्रकार', 'के', 'छोटे-बड़े', 'कृषि,', 'सेवा,', 'व्यपार,उद्योग', 'का', 'पंजीकरण', 'तुरंत', 'अनिवार्य', 'होना', 'चाहिए,', 'साथ', 'उनको', 'रु', '1000', 'से', 'ऊपर', 'के', 'लेन', 'देन', 'को', 'ऑन', 'लाइन/', 'इलेक्ट्रॉनिक', 'माध्यम', 'से', 'करना', 'अनिवार्य', 'होना', 'चाहिये।', 'इस', 'सम्मेलन', 'का', 'उद्घाटन', 'करने', 'पहुंचे', 'पीएम', 'न

### CREATING LIST OF GROUND TRUTH TOKENS FOR EACH CLASS 

In [50]:
ground_truth=[['इस:O', 'बार:O', 'गुरुवार:B-MISC', 'को:O', 'ईसाई:B-ORG', 'समुदाय:I-ORG', 'के:O', 'लोगों:O', 'ने:O', 'यहां:O' , '‘मुर्दों:O', 'की:O', 'ईद'':B-MISC', 'मनाई:O', '।:O'],
['उन्होंने:O', 'कहा:O', ',:O', 'डॉ:B-MISC', 'राजेन्द्र:B-PER', 'प्रसाद:I-PER', ',:O', 'बाबा:B-PER', 'साहेब:I-PER', 'भीमराव:I-PER', 'अंबेडकर:I-PER', ',:O', 'सरदार:B-PER', 'पटेल:I-PER', ',:O', 'मौलाना:B-PER', 'आजाद:I-PER', ',:O', 'सुचेता:B-PER', 'कृपलानी:I-PER', 'और:O', 'अनेक:O', 'अनगिनत:O', 'महापुरुषों:O', 'ने:O', 'प्रत्यक्ष:O', 'और:O', 'अप्रत्यक्ष:O', 'योगदान:O', 'देकर:O', 'ये:O', 'महान:O', 'विरासत:O', 'हमें:O', 'सौंपी:O', 'हैं:O', '।:O'],
['मुझे:O', 'अपने:O', 'विपक्षी:O', 'मित्रों:O', 'के:O', 'तर्क:O', 'इस:O', 'वैज्ञानिक:O', 'के:O', 'निष्कर्ष:O', 'के:O', 'समान:O', 'ही:O', 'प्रतीत:O', 'हो:O', 'रहे:O', 'हैं:O', '....:O'],
['पार्टी:O', 'अगर:O', 'कोई:O', 'हुई:O', 'तो:O', 'कितनी:O', 'फीकी:O', 'लगेगी:O', 'बिना:O', 'गाने:O', 'के:O', '!:O'],
['उनके:O', 'अनुसार:O', 'सरकारी:O', 'अस्पतालों:O', 'की:O', 'तरह:O', 'सरकारी:O', 'स्कूलों:O', 'की:O', 'हालत:O', 'बदतर:O', 'हो:O', 'चुकी:O', 'है:O', '।:O'],
['देश:O', 'में:O', 'सभी:O', 'प्रकार:O', 'के:O', 'छोटे-बड़े:O', 'कृषि:O', ',:O', 'सेवा:O', ',:O', 'व्यपार:O', ',:O', 'उद्योग:O', 'का:O', 'पंजीकरण:O', 'तुरंत:O', 'अनिवार्य:O', 'होना:O', 'चाहिए:O', 'साथ:O', 'उनको:O', 'रु:O', '1000:B-MISC', 'से:O', 'ऊपर:O', 'के:O', 'लेन:O', 'देन:O', 'को:O', 'ऑन लाइन:O', '/:O',  'इलेक्ट्रॉनिक:O', 'माध्यम:O', 'से:O', 'करना:O', 'अनिवार्य:O', 'होना:O', 'चाहिये:O', '।:O'],
['इस:O', 'सम्मेलन:O', 'का:O', 'उद्घाटन:O', 'करने:O', 'पहुंचे:O', 'पीएम:B-MISC', 'मोदी:B-PER', 'ने:O', 'कहा:O', 'कि:O', 'टीबी:B-MISC', 'को:O', '25:B-MISC', 'साल:O', 'पहले:O', 'डब्ल्यूएचओं:B-ORG', 'ने:O', 'एमरजंसी:O', 'घोषित:O', 'किया:O', 'था:O', 'और:O', 'तभी:O', 'से:O', 'इसके:O', 'खिलाफ:O', 'अभियान:O', 'चल:O', 'रहा:O', 'है:O', '।:O'],
['इस:B-MISC', 'वक्त:I-MISC', 'खुदरा:O', 'महंगाई:O', 'दर:O', 'भी:O', 'पांच:B-MISC', 'महीने:O', 'के:O', 'उच्चतम:O', 'स्तर:O', 'पर:O', 'है:O', '।:O'],
['साल:O', '2007:B-MISC', 'के:O', 'क्रिकेट:B-MISC', 'विश्व:I-MISC', 'कप:I-MISC', 'में:O', 'भारत:B-LOC', 'के:O', 'खराब:O', 'प्रदर्शन:O', 'के:O', 'बाद:O', 'भी:O', 'विज्ञापनों:O', 'पर:O', 'खर्च:O', 'हुए:O', '6,500:B-MISC', 'करोड़:I-MISC', 'रुपये:O', 'में:O', 'से:O', 'भी:O', '7-10:B-MISC', 'फीसदी:O', 'की:O', 'हिस्सेदारी:O', 'थी:O', '।:O'],
['हिन्दी:B-MISC', 'के:O', 'शिक्षण-प्रशिक्षण:O', 'के:O', 'लिए:O', 'देश:O', 'के:O', 'कई:O', 'हिस्सों:O', 'के:O', 'साथ-साथ:O', 'विदेशों:O', 'में:O', 'सेंटर:O', 'खोलने:O', 'की:O', 'भी:O', 'योजना:O', 'है:O', '।:O'],
['क्या:O', 'गांधी:B-PER', 'के:O', 'इस:O', 'सुझाव:O', 'पर:O', 'विमर्श:O', 'का:O', 'समय:O', 'आ:O', 'गया:O', 'है:O', '?:O', 'राजनीतिक:O', 'पार्टियां:O', 'हमारी:O', 'संवैधानिक:O', 'आवश्यकता:O', 'नहीं:O', 'हैं:O', '.:O', 'इसलिए:O', 'उम्मीदवारों:O', 'को:O', 'निर्दलीय:O', 'होने:O', 'की:O', 'छूट:O', 'भी:O', 'है:O', '.:O'],
['अनायत:O', 'इसके:O', 'लिए:O', 'विभागाध्यक्ष:O', 'व:O', 'छात्रों:O', 'को:O', 'बधाई:O', 'दी:O', 'है:O', '।:O'],
['इसके:O', 'बाद:O', 'इंटरनेशनल:O', 'अलर्ट:O', 'भी:O', 'जारी:O', 'कर:O', 'दिया:O', 'गया:O', 'था:O', '।:O'],
['राज्य:B-MISC', 'समन्वयक:I-MISC', 'ने:O', 'सीमा:O', 'पुलिस:O', 'के:O', 'अधिकारियों:O', 'को:O', 'भेजे:O', 'एक:B-MISC', 'आदेश:O', 'में:O', '“घोषित:O', 'विदेशियों”:O', 'के:O', 'परिवार:O', 'के:O', 'सदस्यों:O', 'को:O', 'ट्रिब्यूनल:O', 'के:O', 'समक्ष:O', 'प्रस्तुत:O', 'करने:O', 'को:O', 'कहा:O', '.:O'],
['जब:O', 'बड़े-बड़े:O', 'पापी-कपटी:O', 'कामी-क्रोधी:O', 'लोभी:O', 'बदल:O', 'सकते:O', 'हैं:O', 'तो:O', 'हम:O', 'क्यों:O', 'नहीं:O', 'बदल:O', 'सकते:O', '।:O'],
['फिल्म:O', 'में:O', 'नायक:B-MISC', 'हैं:O', 'संजय:B-PER', 'खान:I-PER', 'और:O', 'नायिका:B-MISC', 'मुमताज:B-PER', 'और:O', 'सहनायक:B-MISC', 'फिरोज:B-PER', 'खान:I-PER', '।:O', 'रफी:B-PER', 'साहब:I-PER', 'आशा:B-PER', 'भोंसले:I-PER', 'और:O', 'साथियो:O', 'का:O', 'गाया:O', 'यह:O', 'गीत:O', 'पहले:O', 'रेडियो:O', 'के:O', 'सभी:O', 'केन्द्रों:O', 'से:O', 'बहुत:O', 'सुनवाया:O', 'जाता:O', 'था:O', '।:O', 'अब:O', 'बहुत:O', 'समय:O', 'से:O', 'नहीं:O', 'सुना:O', '।:O', 'इसके:O', 'कुछ:O', 'बोल:O', 'याद:O', 'आ:O', 'रहे:O', 'हैं:O', '-:O'],
['इसमें:O', 'कहा:O', 'गया:O', 'कि:O', '"अकेले:O', 'याचिकाकर्ता:O', 'को:O', 'दंडित:O', 'नहीं:O', 'किया:O', 'जा:O', 'रहा:O', 'है:O', ',:O', 'बल्कि:O', 'आपराधिक:O', 'कार्यवाही:O', 'के:O', 'कारण:O', 'उसका:O', 'पूरा:O', 'परिवार:O', 'अत्यंत:O', 'पीड़ित:O', 'हुआ:O', 'है:O', '।:O'],
['हम:O', 'आपकी:O', 'सेवा:O', 'में:O', 'अपने:O', 'शरीर:O', 'के:O', 'बारे:O', 'में:O', 'बहुत:O', 'ही:O', 'साधारण:O', 'सी:O', 'बात:O', 'पेश:O', 'करते:O', 'हैं:O', '।:O'],
['बायर्न:B-LOC', 'म्यूनिख:I-LOC', 'में:O', 'इस:O', 'सीजन:O', 'लोन:O', 'पर:O', 'शामिल:O', 'हुए:O', 'ब्राजील:B-LOC', 'के:O', 'खिलाड़ी:O', 'फिलिप:B-PER', 'कोटिन्हो:I-PER', 'को:O', 'कहना:O', 'है:O', 'कि:O', 'वह:O', 'जर्मन:B-ORG', 'क्लब:I-ORG', 'में:O', 'लंबे:O', 'समय:O', 'तक:O', 'रहकर:O', 'कई:O', 'ट्रॉफी:O', 'जीतना:O', 'चाहते:O', 'हैं:O', '।:O'],
['गोवर्धन:B-MISC', 'पूजा:I-MISC', 'में:O', 'भगवान:B-MISC', 'को:O', 'छप्पन:B-MISC', 'भोग:O', 'का:O', 'प्रसाद:O', 'चढ़ाया:O', 'जाता:O', 'हैं:O', '।:O'],
['कार्यवाहक:B-MISC', 'कलक्टर:I-MISC', 'ने:O', 'आदेश:O', 'जारी:O', 'करते:O', 'हुए:O', 'विभागीय:O', 'निर्देशानुसार:O', 'निरीक्षण:O', 'एवं:O', 'व्यवस्था:O', 'का:O', 'मूल्यांकन:O', 'नहीं:O', 'करने:O', 'पर:O', 'छात्रावास:O', 'में:O', 'अनियमितताएं:O', 'व्याप्त:O', 'होने:O', 'के:O', 'कारण:O', 'सहायक:B-MISC', 'निदेशक:I-MISC', 'दिलीप:B-PER', 'रोकडिय़ा:B-PER', 'को:O', 'नोटिस:O', 'जारी:O', 'कर:O', 'तीन:B-MISC', 'दिन:O', 'में:O', 'जवाब:O', 'देने:O', 'को:O', 'कहा:O', 'है:O', '।:O'],
['हनीसकल:B-MISC', 'की:O', 'खेती:O', 'क्षेत्र:O', 'मुख्य:O', 'रूप:O', 'से:O', 'शेडोंग:B-LOC', 'शानक्सी:B-LOC', 'हेनान:B-LOC', 'हेबै:B-LOC', 'हुबेई:B-LOC', 'जियांग्सी:B-LOC', 'गुआंग्डोंग:B-LOC', 'और:O', 'अन्य:O', 'स्थानों:O', 'पर:O', 'केंद्रित:O', 'हैं:O', '।:O'],
['इसके:O', 'साथ:O', 'ही:O', 'चीन:B-ORG', 'सक्रिय:I-ORG', 'सेवा:I-ORG', 'में:O', 'विमानवाही:B-MISC', 'पोत:I-MISC', 'रखने:O', 'वाला:O', 'दुनिया:O', 'का:O', '10वां:B-MISC', 'और:O', 'संयुक्त:B-ORG', 'राष्ट्र:I-ORG', 'सुरक्षा:I-ORG', 'परिषद:I-ORG', 'के-O', 'स्थायी:O', 'पांच:B-MISC', 'सदस्यों:O', 'में:O', 'अंतिम:O', 'देश:O', 'बन:O', 'गया:O', 'है:O', '।:O'],
['संसारके:O', 'दुःख:O', 'मुझे:O', 'प्राणप्रिय:O', 'लगते:O', 'हैं:O', ',:O', 'उन्हें:O', 'मैं:O', 'कैसे:O', 'छोड:O', ',:O', 'सकता:O', 'हूँ:O', '?:O', 'उन:O', 'दुःखोंको:O', 'तो:O', 'मैंने:O', 'ही:O', 'बुलाया:O', 'है:O', '.:O'],
['उन्होंने:O', 'इस:O', 'बाबत:O', 'पिछले:O', 'दिनों:O', 'फेसबुक:B-ORG', 'पर:O', 'जो:O', 'कुछ:O', 'लाइनें:O', 'लिखकर:O', 'अपने:O', 'जानने:O', 'वालों:O', 'को:O', 'सूचित:O', 'किया:O', 'वह:O', 'इस:O', 'प्रकार:O', 'है:O', '::O']]

In [51]:
gt = {'O' : [],'B_PER' : [],'I_PER' : [],'B_LOC' : [],'I_LOC' : [],'B_ORG' : [],'I_ORG' : [],'B_MISC' : [],'I_MISC' : []}

In [52]:
for sublist in ground_truth:
    for item in sublist:
        temp = item.split(':')
        token = temp[0]
        if(len(temp)>1):
            tag = temp[1]
            if(tag=='O'): gt['O'].append(token)
            elif(tag=='B-PER'): gt['B_PER'].append(token)
            elif(tag=='I-PER'): gt['I_PER'].append(token)
            elif(tag=='B-LOC'): gt['B_LOC'].append(token)
            elif(tag=='I-LOC'): gt['I_LOC'].append(token)
            elif(tag=='B-ORG'): gt['B_ORG'].append(token)
            elif(tag=='I-ORG'): gt['I_ORG'].append(token)
            elif(tag=='B-MISC'): gt['B_MISC'].append(token)
            elif(tag=='I-MISC'): gt['I_MISC'].append(token)
for key in gt.keys():
    print(key, ":", gt[key])

O : ['इस', 'बार', 'को', 'के', 'लोगों', 'ने', 'यहां', '‘मुर्दों', 'की', 'मनाई', '।', 'उन्होंने', 'कहा', ',', ',', ',', ',', ',', 'और', 'अनेक', 'अनगिनत', 'महापुरुषों', 'ने', 'प्रत्यक्ष', 'और', 'अप्रत्यक्ष', 'योगदान', 'देकर', 'ये', 'महान', 'विरासत', 'हमें', 'सौंपी', 'हैं', '।', 'मुझे', 'अपने', 'विपक्षी', 'मित्रों', 'के', 'तर्क', 'इस', 'वैज्ञानिक', 'के', 'निष्कर्ष', 'के', 'समान', 'ही', 'प्रतीत', 'हो', 'रहे', 'हैं', '....', 'पार्टी', 'अगर', 'कोई', 'हुई', 'तो', 'कितनी', 'फीकी', 'लगेगी', 'बिना', 'गाने', 'के', '!', 'उनके', 'अनुसार', 'सरकारी', 'अस्पतालों', 'की', 'तरह', 'सरकारी', 'स्कूलों', 'की', 'हालत', 'बदतर', 'हो', 'चुकी', 'है', '।', 'देश', 'में', 'सभी', 'प्रकार', 'के', 'छोटे-बड़े', 'कृषि', ',', 'सेवा', ',', 'व्यपार', ',', 'उद्योग', 'का', 'पंजीकरण', 'तुरंत', 'अनिवार्य', 'होना', 'चाहिए', 'साथ', 'उनको', 'रु', 'से', 'ऊपर', 'के', 'लेन', 'देन', 'को', 'ऑन लाइन', '/', 'इलेक्ट्रॉनिक', 'माध्यम', 'से', 'करना', 'अनिवार्य', 'होना', 'चाहिये', '।', 'इस', 'सम्मेलन', 'का', 'उद्घाटन', 'करने', 'पहुंचे', 'ने', '

In [53]:
def score_cal(ground_truth, m_output):
    true_positives = 0.0
    false_positives = 0.0
    false_negatives = 0.0
    true_positives += len(set(ground_truth) & set(m_output))
    false_positives += len(set(m_output) - set(ground_truth))
    false_negatives += len(set(ground_truth) - set(m_output))
    if (true_positives + false_positives) != 0: precision = true_positives / (true_positives + false_positives)
    else : precision = 0
    if (true_positives + false_negatives) != 0: recall = true_positives / (true_positives + false_negatives)
    else : recall = 0
    if (precision + recall) != 0: f_score = 2 * (precision * recall) / (precision + recall)
    else : f_score = 0
    return [precision, recall, f_score]

### MACRO F1-SCORE AND FINAL METRICS FOR EACH CLASS

In [54]:
metric={'O' : [],'B_PER' : [],'I_PER' : [],'B_LOC' : [],'I_LOC' : [],'B_ORG' : [],'I_ORG' : [],'B_MISC' : [],'I_MISC' : []}
total = 0
for key in gt.keys():
    output = score_cal(gt[key],  pd[key])
    metric[key] = output
    total += output[2]
avg_f_score = total / len(metric)
print("MACRO F1-SCORE FOR 25 SENTENCES USING IndicNER:\n",avg_f_score)
print("CLASSWISE PRECISION, RECALL AND F1-SCORE FOR 25 SENTENCES:\n")
for key in metric.keys():
    print(key ," : " ,metric[key])

MACRO F1-SCORE FOR 25 SENTENCES USING IndicNER:
 0.2833128469036034
CLASSWISE PRECISION, RECALL AND F1-SCORE FOR 25 SENTENCES:

O  :  [0.7648809523809523, 0.8954703832752613, 0.8250401284109148]
B_PER  :  [0.3684210526315789, 0.4666666666666667, 0.4117647058823529]
I_PER  :  [0.16666666666666666, 0.18181818181818182, 0.17391304347826086]
B_LOC  :  [0.2222222222222222, 0.2, 0.2105263157894737]
I_LOC  :  [0, 0.0, 0]
B_ORG  :  [0.375, 0.5, 0.42857142857142855]
I_ORG  :  [0.6, 0.42857142857142855, 0.5]
B_MISC  :  [0, 0.0, 0]
I_MISC  :  [0, 0.0, 0]
